# Exercício 2

Uma companhia aérea regional pode comprar seu combustível para jato a partir de qualquer um dentre três fornecedores. As necessidades da companhia aérea para o mês entrante em cada um dos três aeroportos emq ue ela opera são: 100.000 galões no aeroporto 1; 180.000 galões no aeroporto 2 e 300.000 galões no aeroporto 3. Cada fornecedor pode abastecer cada um dos aeroportos de acordo com os preços (em $ por galão) dados no seguinte quadro:

|                  	| **Aeroporto 1** 	| **Aeroporto 2** 	| **Aeroporto 3** 	|
|:----------------:	|:---------------:	|:---------------:	|:---------------:	|
| **Fornecedor 1** 	|        92       	|        89       	|        90       	|
| **Fornecedor 2** 	|        91       	|        91       	|        95       	|
| **Fornecedor 3** 	|        87       	|        90       	|        92       	|

Cada fornecedor, contudo, está limitado pelo número total de galões que ele pode abastecer por mês. Estas capacidades são 320.000 galões para o fornecedor 1, 270.000 galões para o fornecedor 2 e 150.000 galões para o fornecedor 3. Determine a política de aquisição que suprirá as necessidades da companhia em cada aeroporto a um custo total mínimo.

# Solução

Inicialmente, vamos elaborar uma tabela que agregue todas as informações contidas na situação.

|                  	| **Aeroporto 1** 	| **Aeroporto 2** 	| **Aeroporto 3** 	| **Oferta** 	|
|:----------------:	|:---------------:	|:---------------:	|:---------------:	|:----------:	|
| **Fornecedor 1** 	|        92       	|        89       	|        90       	|   320000   	|
| **Fornecedor 2** 	|        91       	|        91       	|        95       	|   270000   	|
| **Fornecedor 3** 	|        87       	|        90       	|        92       	|   150000   	|
|    **Demanda**   	|      100000     	|      180000     	|      300000     	|            	|

## Conjuntos de Iteração

$I$: conjunto de fornecedores, indexado por $i$.

$J$: conjunto de aeroportos, indexado por $j$.

## Parâmetros do modelo

$c_{ij}$: custo unitário de transporte, saindo do fornecedor $i$ para o aeroporto $j$.

$o_{i}$: capacidade (oferta) máxima associada a cada fornecedor $i$.

$d_{j}$: demanda associada a cada aeroporto $j$.

## Variáveis de Decisão

$x_{ij}$: quantidade transportada do fornecedor $i$ para o aeroporto $j$.

## Função Objetivo

$min \ Z = \sum_{i \in I}\sum_{j \in J} \ x_{ij}c_{ij}$

## Restrições

$\sum_{i \in I} x_{ij} = d_{j}, \ \forall j \in J$: a soma de todos os fornecimentos $i$ para um dado aeroporto $j$ deve ser igual à demanda do aeroporto $j$, para todos os aeroportos.

$\sum_{j \in J} x_{ij} \le o_{i},\ \forall i \in I$: o somatório de todas as demandas $j$ para um dado fornecedor $i$ deve ser menor ou igual à capacidade $i$ daquele fornecedor, para todos os fornecedores.

$x_{ij} \ge 0, \ x_{ij} \in  \mathbb{Z}$


In [41]:
# --- Imports --- #
import pyomo.environ as pyo
import pandas as pd

In [42]:
# --- Declaração de dados de entrada --- #
# Custo de transporte do fornecedor i para o cliente j.
custos_transporte = {('F1', 'A1'): 92, ('F1', 'A2'): 89, ('F1', 'A3'): 90,
                     ('F2', 'A1'): 91, ('F2', 'A2'): 91, ('F2', 'A3'): 95,
                     ('F3', 'A1'): 87, ('F3', 'A2'): 90, ('F3', 'A3'): 92}
ofertas = {'F1': 320000, 'F2': 270000, 'F3': 150000} # Oferta de cada fornecedor
demandas = {'A1': 100000, 'A2': 180000, 'A3': 300000} # Demanda de cada cliente

In [43]:
# --- Declaração do Modelo Matemático --- #
modelo = pyo.ConcreteModel()

In [44]:
# --- Declaração dos Conjuntos de Iteração --- #
modelo.I = pyo.Set(initialize=ofertas.keys()) # Conjunto de ofertas
modelo.J = pyo.Set(initialize=demandas.keys()) # Conjunto de demandas

In [45]:
# --- Declaração de Parâmetros --- #
# Custo de transporte do fornecedor i para o cliente j
modelo.custo_transporte = pyo.Param(modelo.I, modelo.J, initialize=custos_transporte)
# Oferta do fornecedor i
modelo.oferta = pyo.Param(modelo.I, initialize=ofertas)
# Demanda do cliente j
modelo.demanda = pyo.Param(modelo.J, initialize=demandas)

In [46]:
# --- Declaração das Variáveis de Decisão --- #
modelo.x = pyo.Var(modelo.I, modelo.J, domain=pyo.NonNegativeIntegers) # x_ij = quantidade de produtos transportados do fornecedor i para o cliente j

In [47]:
# --- Declaração da Função Objetivo --- #
# Regra de função objetivo
def fo(modelo):
    return sum(modelo.custo_transporte[i, j] * modelo.x[i, j]
               for i in modelo.I
               for j in modelo.J
)
modelo.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize) # Objetivo de função objetivo

In [48]:
# --- Declaração das Restrições --- #
# Restrição de atendimento da demanda
def restricao_demanda(modelo, j):
    '''
    Recebe um objeto de modelo Pyomo e um índice j do conjunto J e retorna a expressão de restrição
    de atendimento da demanda do cliente j.
    '''
    return sum(modelo.x[:, j]) == modelo.demanda[j]
# ---
modelo.rest_demanda = pyo.Constraint(modelo.J, rule=restricao_demanda) # Restrição de atendimento da demanda
# --- #
# Restrição de capacidade máxima do fornecedor
def restricao_capacidade(modelo, i):
    '''
    Recebe um objeto de modelo Pyomo e um índice i e retorna a expressão de restrição
    de capacidade máxima do fornecedor i.
    '''
    return sum(modelo.x[i, :]) <= modelo.oferta[i]
# ---
modelo.rest_capacidade = pyo.Constraint(modelo.I, rule=restricao_capacidade) # Restrição de capacidade máxima do fornecedor i.

In [49]:
# --- Declaração do Solver --- #
solver = pyo.SolverFactory('gurobi') # Criação de um objeto solver
solver.solve(modelo) # Resolução do modelo

{'Problem': [{'Name': 'x1', 'Lower bound': 51990000.0, 'Upper bound': 51990000.0, 'Number of objectives': 1, 'Number of constraints': 6, 'Number of variables': 9, 'Number of binary variables': 0, 'Number of integer variables': 9, 'Number of continuous variables': 0, 'Number of nonzeros': 18, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.002000093460083008', 'Error rc': 0, 'Time': 0.3222520351409912}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [50]:
# --- Extração dos Resultados --- #
dados_transporte = [{'de': i, 'para': j, 'quantidade': val}
                    for (i, j), val in modelo.x.extract_values().items()]
# ---
resultados = pd.DataFrame(dados_transporte).pivot(index='de', columns='para', values='quantidade')
resultados.to_excel('_ex_02_resultados_modelo_matematico.xlsx')

In [51]:
print(f'Função Objetivo: {modelo.objetivo()}') # Valor da função objetivo

Função Objetivo: 51990000.0


In [52]:
dados_transporte

[{'de': 'F1', 'para': 'A1', 'quantidade': -0.0},
 {'de': 'F1', 'para': 'A2', 'quantidade': 20000.0},
 {'de': 'F1', 'para': 'A3', 'quantidade': 300000.0},
 {'de': 'F2', 'para': 'A1', 'quantidade': -0.0},
 {'de': 'F2', 'para': 'A2', 'quantidade': 110000.0},
 {'de': 'F2', 'para': 'A3', 'quantidade': -0.0},
 {'de': 'F3', 'para': 'A1', 'quantidade': 100000.0},
 {'de': 'F3', 'para': 'A2', 'quantidade': 50000.0},
 {'de': 'F3', 'para': 'A3', 'quantidade': 0.0}]